# LLM "No Hint" Migration 
This notebook demonstrates how an LLM (using Kai's ModelProvider) performs when presented with a Spring Security migration issue without a hint.

Installing pre requisites and configuring Kai with `GPT-3.5-turbo`

In [ ]:
%pip uninstall kai -y
%pip install --no-cache-dir git+https://github.com/konveyor/kai.git@main
%pip install python-dotenv

In [2]:
from IPython.display import display, Markdown
from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders
from dotenv import load_dotenv
import os
load_dotenv(override=True) 

# Initialize the model provider using Llama-3.1-8B-Instruc via OpenAI
model = ModelProvider.from_config(KaiConfigModels(
    provider=SupportedModelProviders.CHAT_OPENAI,
    args={"model": "meta-llama/Llama-3.1-8B-Instruct",
        "base_url": "https://llama-3-1-8b-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443/v1",
        "api_key": os.getenv("OPENAI_API_KEY"),},
))

# Async rendering function for displaying the response
async def rendered_llm_call(prompt: str):
    response = await model.ainvoke_llm(prompt)
    display(Markdown(response.content))
    return response



/Users/sraghuna/local_dev/kai-notebook-solution-server/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Below is the snippet we are trying to migrate from springboot2 to springboot3. This code uses the discouraged `configure(AuthenticationManagerBuilder)` method inside a `WebSecurityConfigurerAdapter`, both of which are deprecated in Spring Security 5.7+ and removed in Spring Boot 3.

Our goal is to assess how well the LLM refactors this configuration without a hint, and whether it can replace deprecated patterns with the right approach.

In [3]:
before_code = """\
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

@Configuration
@EnableWebSecurity
public class SecurityConfig extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        auth.inMemoryAuthentication()
            .withUser("user")
            .password(passwordEncoder().encode("password"))
            .roles("USER");
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
"""

This rule flags the use of `configure(AuthenticationManagerBuilder)`. In Spring Boot 3, developers should instead use a `UserDetailsService` and `PasswordEncoder` bean for authentication configuration.

In [4]:
analysis_rule=""""\
    
  ruleID: spring-security-replace-authenticationmanagerbuilder-00001
  description: configure(AuthenticationManagerBuilder) override is discouraged since Spring Security 5.7.
  category: mandatory
  effort: 2
  labels:
    - konveyor.io/source=spring-boot2
    - konveyor.io/target=spring-boot3+
  message: |
    Overriding `configure(AuthenticationManagerBuilder)` is **discouraged** in Spring Security 5.7.
    Instead, define a `UserDetailsService` and `PasswordEncoder` bean.
    ```
  when:
    java.referenced:
      location: METHOD
      pattern: configure(org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder) *
    """

In [5]:
prompt = f"""
The following java code needs to be migrated from springboot2 to springboot3. 

```java
{before_code}
```
The issue has been identified by an analysis rule

{analysis_rule}

Can you provide a refactored version of this code that is compatible with springboot3?
Also, create an hint based on your suggestions.

"""

In [6]:

response = await rendered_llm_call(prompt)

Here's the refactored version of the code that is compatible with Spring Boot 3:

```java
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.authentication.configuration.AuthenticationConfiguration;
import org.springframework.security.config.annotation.web.builders.HttpSecurity;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;
import org.springframework.security.provisioning.InMemoryUserDetailsManager;

@Configuration
@EnableWebSecurity
public class SecurityConfig {
    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }

    @Bean
    public InMemoryUserDetailsManager userDetailsService(PasswordEncoder passwordEncoder) {
        return new InMemoryUserDetailsManager(
                java.util.Arrays.asList(
                        new org.springframework.security.core.userdetails.User(
                                "user",
                                passwordEncoder.encode("password"),
                                java.util.Arrays.asList("USER")
                        )
                )
        );
    }

    @Bean
    public org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter securityConfigurerAdapter(AuthenticationConfiguration authenticationConfiguration) throws Exception {
        return new org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter() {
            @Override
            protected void configure(HttpSecurity http) throws Exception {
                http.authorizeRequests()
                        .anyRequest().authenticated();
            }

            @Override
            protected void configure(org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder auth) throws Exception {
                auth.userDetailsService(userDetailsService(passwordEncoder()));
            }
        };
    }
}
```

**Hint:**

To fix the analysis rule, you need to define a `UserDetailsService` and `PasswordEncoder` bean. 

1. Replace the `configure(AuthenticationManagerBuilder)` method with a `UserDetailsService` bean.
2. Define a `PasswordEncoder` bean.
3. In the `configure(HttpSecurity)` method, use the `userDetailsService` bean to configure the authentication manager.
4. Use the `passwordEncoder` bean to encode the password.

Note: The `WebSecurityConfigurerAdapter` is deprecated since Spring Security 5.7. It's recommended to use the `SecurityConfigurerAdapter` instead. However, in this example, we're using the deprecated class for simplicity.

### Evaluation

| Criteria                                                                 | Result         |
|--------------------------------------------------------------------------|----------------|
| Removed deprecated `configure(AuthenticationManagerBuilder)`            |  No           |
| Replaced with `UserDetailsService` and `PasswordEncoder` bean           | Partially Yes |
| Removed `WebSecurityConfigurerAdapter`                                   |  No           |
| The output compiles                                                      | <todo>          |
| Preserves behavior (in-memory auth, user with role)                      | No           |

